In [3]:
import torch
from transformers import ElectraModel, ElectraTokenizer
import pandas as pd

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch

import os

from sklearn.model_selection import train_test_split

import time

## 데이터셋 호출
dataset 디렉토리에 있는 csv파일들의 path 생성합니다.

In [18]:
file_list = os.listdir('./dataset_bk')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./dataset_bk', file_name)
    file_path.append(tmp_path)
print(file_path)

['./dataset_bk/ulsan_2022_cause-effect_v2.csv', './dataset_bk/port_cause_effect_v2.csv', './dataset_bk/2019-2021_ulsan_cause_effect_v2.csv', './dataset_bk/eshc_cause-effect_v2.csv']


csv 파일들을 순차적으로 읽어옵니다. data_list에는 토큰들이, label_list에는 각 토큰의 label이 담겨있습니다..
아래 예시 참고해주세요.

In [19]:
data_list = []
label_list = []
for path in file_path:
    tmp_data = []
    tmp_label = []
    f = open(path, 'r', encoding='utf-8-sig')
    lines = f.readlines()
    f.close()
    for line in lines:
        # print(line)
        # print(''.join(line.split(',')[:-1]))
        # print(line.split(',')[-1])
        # print(line.split(',')[-1][:-1])
        if line == ',\n':
            data_list.append(tmp_data)
            label_list.append(tmp_label)
            tmp_data = []
            tmp_label = []
        else:
            tmp_data.append(''.join(line.split(',')[:-1]))
            tmp_label.append(line.split(',')[-1][:-1])

In [20]:
print(len(data_list))
print(len(label_list))

3869
3869


In [26]:
data_list[100], label_list[100]

(['2022',
  '##년',
  '5',
  '##월',
  '30',
  '##일',
  '오후',
  '16',
  '##시',
  '4',
  '0',
  '##분',
  '##경',
  '경상남도',
  '함안',
  '군',
  '소재',
  '제조업',
  '공장',
  '내',
  '##에',
  '##서',
  '분',
  '##체',
  '도장',
  '실',
  '내',
  '세척',
  '##조',
  '##의',
  '히터',
  '##봉',
  '교체',
  '작업',
  '을',
  '위해',
  '드럼',
  '##펌',
  '##프로',
  '세척',
  '##액',
  '##을',
  '빈',
  '드럼통',
  '에',
  '이송',
  '##하',
  '##는',
  '작업',
  '중',
  '화재',
  '·',
  '폭발',
  '##가',
  '발생',
  '##하',
  '##였',
  '##다',
  '.',
  '이',
  '사고',
  '##로',
  '작업자',
  '1',
  '##명',
  '##이',
  '사망',
  '##하',
  '##고',
  '1',
  '##명',
  '##이',
  '부상',
  '##을',
  '입',
  '##었',
  '##다',
  '.'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'O',
  'O

문장을 구성하는 토큰들의 시작점에 문장의 시작을 알리는 CLS 토큰을 추가합니다. 마지막 지점에는 문장의 끝을 알리는 SEP 토큰을 추가합니다. 각 문장을 구성하는 토큰의 개수가 동일해야만 병렬 연산이 가능하므로 [PAD] 토큰을 이용해서 각 문장의 길이(토큰의 개수)를 256개로 통일합니다. 이 때 BERT에서 PAD 토큰에 대해 attention을 수행하지 않도록 mask 리스트(변수명: input_mask)도 같이 생성합니다.

In [22]:
tokenizer = ElectraTokenizer.from_pretrained('./tokenizer')

init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
sep_token = tokenizer.sep_token

print(init_token, pad_token, unk_token, sep_token)

[CLS] [PAD] [UNK] [SEP]


In [23]:
input_tokens = []
input_mask = []
input_labels = []
max_length = 256
num = len(data_list)
for i in range(num):
    data = data_list[i]
    labels = label_list[i]
    tmp_tokens = [init_token] + data
    tmp_labels = ['O'] + labels
    if len(tmp_tokens) < max_length-1:
        tmp_tokens += [sep_token]
        tmp_labels += ['O']
        tmp_mask = [1] * len(tmp_tokens)
        for _ in range(max_length-len(tmp_tokens)):
            tmp_tokens.append(pad_token)
            tmp_labels.append('PAD')
            tmp_mask.append(0)
        input_tokens.append(tmp_tokens)
        input_labels.append(tmp_labels)
        input_mask.append(tmp_mask)
    elif len(tmp_tokens) > max_length-1:
        tmp_tokens = tmp_tokens[:max_length-1] + [sep_token]
        tmp_labels = tmp_labels[:max_length-1] + ['O']
        tmp_mask = [1] * max_length
        input_tokens.append(tmp_tokens)
        input_labels.append(tmp_labels)
        input_mask.append(tmp_mask)
    elif len(tmp_tokens) == max_length-1:
        tmp_tokens += [sep_token]
        tmp_labels += ['O']
        tmp_mask = [1] * max_length
        input_tokens.append(tmp_tokens)
        input_labels.append(tmp_labels)
        input_mask.append(tmp_mask)
        

In [24]:
print(input_labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'C', 'C', 'C', 'O', 'O', 'O', 'O', 'O', 'CE1', 'CE1', 'CE1', 'CE1', 'O', 'O', 'O', 'O', 'O', 'O', 'CE2', 'CE2', 'CE2', 'CE2', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'E', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD

In [9]:
# num = len(input_tokens)
# for i in range(num):
#     curr_tokens = input_tokens[i]
#     curr_label = input_labels[i]
#     curr_mask = input_mask[i]
    
#     count_tokens, count_label, count_mask = 0,0,0
#     for count, token in enumerate(curr_tokens):
#         if token == '[PAD]':
#             break
#     count_tokens = count
#     for count, label in enumerate(curr_label):
#         if label == 'PAD':
#             break
#     count_label = count
#     for count, mask in enumerate(curr_mask):
#         if mask == 0:
#             break
#     count_mask = count

In [25]:
input_pair = []
num = len(input_tokens)
for i in range(num):
    tmp_token_idx = input_tokens[i]
    tmp_mask = input_mask[i]
    tmp_label_idx = input_labels[i]
    input_pair.append((tmp_token_idx, tmp_mask, tmp_label_idx))

In [11]:
# input_pair[3868][2]

데이터셋을 8:2 비율로 훈련 데이터셋과 검증 데이터셋으로 분할합니다. 데이터셋의 양이 적어 테스트셋은 생성하지 않았습니다.

In [12]:
train, valid = train_test_split(input_pair, test_size=0.2)

In [57]:
test = ['[CLS]', "'", '2019', '##년', '11', '##월', '16', '##일', '오전', '11', '##시', '6', '##분', '##경', '평창', '군', '한', '퇴비', '공장', '에서', '경사', '##진', '도로', '##에', '정차', '중', '이', '##던', '페이', '로더', '(', '중장비', "'", '""', "'", '가', '밀려', '내려가', '##는', '것', '##을', '막', '##으', '##려', '##다', '바퀴', '##에', '깔려', '사망', '##하', '##였', '##다',]
str(test)

'[\'[CLS]\', "\'", \'2019\', \'##년\', \'11\', \'##월\', \'16\', \'##일\', \'오전\', \'11\', \'##시\', \'6\', \'##분\', \'##경\', \'평창\', \'군\', \'한\', \'퇴비\', \'공장\', \'에서\', \'경사\', \'##진\', \'도로\', \'##에\', \'정차\', \'중\', \'이\', \'##던\', \'페이\', \'로더\', \'(\', \'중장비\', "\'", \'""\', "\'", \'가\', \'밀려\', \'내려가\', \'##는\', \'것\', \'##을\', \'막\', \'##으\', \'##려\', \'##다\', \'바퀴\', \'##에\', \'깔려\', \'사망\', \'##하\', \'##였\', \'##다\']'

In [60]:
temp = []
for e in train[0][0]:
    encoded = tokenizer.convert_tokens_to_ids(str(e))
    temp.append(encoded)
temp

[2,
 11,
 9426,
 4556,
 6307,
 4501,
 6427,
 4366,
 6853,
 6307,
 4114,
 26,
 4473,
 4158,
 9937,
 2103,
 3757,
 35438,
 7130,
 6215,
 12576,
 4189,
 7010,
 4073,
 18251,
 3330,
 3240,
 4820,
 9437,
 35001,
 12,
 35670,
 11,
 1,
 11,
 2010,
 9735,
 10311,
 4034,
 2048,
 4292,
 2629,
 4154,
 4228,
 4176,
 11309,
 4073,
 14767,
 6904,
 4279,
 4737,
 4176,
 18,
 11,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

pytorch에서 데이터를 받아들일 수 있도록 커스텀 데이터셋을 생성합니다. 이 때 문장을 구성하는 토큰들을 고유한 인덱스 번호로 변환합니다. 마찬가지로 레이블도 인덱스 번호로 변환합니다.

In [13]:
class CustomDataset(Dataset): 
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx][0]
        masks = self.data[idx][1]
        labels = self.data[idx][2]
        
        token_ids = []
        for token in tokens:
            ids = tokenizer.convert_tokens_to_ids(str(token))
            token_ids.append(ids)

        label_ids = []
        for label in labels:
            if label == 'PAD':
                label_ids.append(0)
            elif label == 'O':
                label_ids.append(1)
            elif label == "C":
                label_ids.append(2)
            elif label == "E":
                label_ids.append(4)
            else:
                label_ids.append(3)
                
        return (torch.LongTensor(token_ids), torch.LongTensor(masks), torch.LongTensor(label_ids))

In [14]:
trainset = CustomDataset(train, tokenizer)
validset = CustomDataset(valid, tokenizer)

배치 크기가 128인 데이터로더를 생성합니다.

In [15]:
train_loader = DataLoader(trainset, batch_size = 128, shuffle = True)
valid_loader = DataLoader(validset, batch_size = 128, shuffle = True)

모델을 생성합니다.

In [16]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, mask):
        
        #text = [batch size, sent len]
        embedded = self.dropout(self.bert(text, mask)[0])
        #embedded = [batch size, seq len, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        
        return predictions

In [17]:
bert = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
print(len(tokenizer))


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


36223


In [18]:
OUTPUT_DIM = 5
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

Embedding(36223, 768)

In [41]:
model.bert.config.to_dict()['hidden_size']

768

In [36]:
bert_result = model.bert(trainset[0][0].unsqueeze(0), trainset[0][1].unsqueeze(0))

torch.Size([1, 256, 768])

In [40]:
bert_result.last_hidden_state.shape
# bert_result.last_hidden_state[0].shape

torch.Size([1, 256, 768])

In [43]:
tagger_result = model(trainset[0][0].unsqueeze(0), trainset[0][1].unsqueeze(0))

In [44]:
tagger_result.shape

torch.Size([1, 256, 5])

모델을 학습시키기 위한 옵티마이저와 목적함수(cross entropy)를 호출합니다.

In [42]:
LEARNING_RATE = 5e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [43]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

GPU 연산을 위해 cuda를 호출합니다. 이 때 복수의 연산장치(GPU)가 있을 경우 모든 연산장치를 활용하여 연산의 속도를 높이도록 설정합니다.

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))
    # torch.multiprocessing.set_start_method('spawn', force=True)
model = model.to(device)
criterion = criterion.to(device)

정확도를 계산하기 위한 함수를 생성합니다.

In [49]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

모델을 훈련시키기 위한 함수입니다.

In [64]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch[0].to(device)
        mask = batch[1].to(device)
        tags = batch[2].to(device)

        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        # print(text)
        # print()
        # print(mask)
        # print()
        # print(tags)

        predictions = model(text, mask)

        # print()
        # print(predictions)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1]) # shape(batchsize, length, ouputdim) --> shape(batchsize, length*outputdim)
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

모델의 성능을 평가하기 위한 함수입니다.

In [61]:

def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

에폭마다 걸리는 시간을 계산하기 위한 함수입니다.

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

실질적으로 모델을 학습시킵니다. 에폭마다 검증데이터셋에서 성능을 평가하여 검증데이터셋에서 가장 좋은 성능을 보여줬을 때의 모델의 weight를 저장합니다.

In [65]:
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, 0)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, 0)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

tensor([[    2, 19505,  4556,  ...,     0,     0,     0],
        [    2, 35807,  3130,  ...,     0,     0,     0],
        [    2,  7285, 12121,  ...,     0,     0,     0],
        ...,
        [    2, 15227,  9029,  ...,     0,     0,     0],
        [    2,  9843,  7402,  ...,     0,     0,     0],
        [    2,  6932,  4556,  ...,     0,     0,     0]])

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


: 

: 

학습된 모델을 이용하여 예시 문장으로 inference 합니다.

In [54]:
model.load_state_dict(torch.load("tut2-model.pt"))
model.eval()
with torch.no_grad():
    idx_to_label = {0:"pad", 1:"O", 2:"C", 3:"CE", 4:"E"}
    sample = '2013년 11월 18일 인천항 4부두에서 컨테이너 작업 중 핸들러 집게가 풀리면서 피해자 끼임. 허리에 경상을 입음'
    words = tokenizer.tokenize(sample)
    tokens = tokenizer(sample)["input_ids"]
    mask = tokenizer(sample)['attention_mask']
    tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    tensor_mask = torch.LongTensor(mask).unsqueeze(0).to(device)
    logit = model(tensor, tensor_mask)
    pred = torch.argmax(logit, dim=-1).squeeze().detach().cpu().tolist()
    for i in range(len(words)):
        print(words[i], idx_to_label[pred[i+1]])

2013 O
##년 O
11 O
##월 O
18 O
##일 O
인천 O
항 O
4 O
부두 O
에서 O
컨테이너 C
작업 C
중 O
핸들러 CE
집게 CE
##가 CE
풀리 CE
##면 CE
##서 CE
피해자 O
끼 CE
##임 CE
. O
허리 O
##에 O
경상 E
##을 O
입 O
##음 O


검증 데이터셋에서 5%의 데이터를 임의로 추출하여 inference 한 다음 그 결과를 csv파일로 저장합니다.

In [55]:
import random
import numpy as np

num = len(valid)
random_choice = np.random.choice(num, int(num*0.05), replace=False).tolist()
selected_sample = []

for i in random_choice:
    selected_sample.append(valid[i])
    
print(len(selected_sample))

38


In [56]:
model.eval()
with torch.no_grad():
    sample_results = []
    for idx, _ in enumerate(selected_sample):
        item = selected_sample[idx]
        token_ids = list(map(lambda x:tokenizer.convert_tokens_to_ids(x), item[0]))
        input_tensor = torch.LongTensor(token_ids).unsqueeze(0).to(device)
        mask_tensor = torch.LongTensor(item[1]).unsqueeze(0).to(device)
        output = model(input_tensor, mask_tensor)
        pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
        sample_results.append(pred)

In [57]:
f = open('sample_result.csv', 'w', encoding='utf-8-sig')
idx_to_label = {0:"pad", 1:"O", 2:"C", 3:"CE", 4:"E"}
for idx, item in enumerate(selected_sample):
    all_tokens = item[0]
    if '[PAD]' not in all_tokens:
        continue
    pad_loc = all_tokens.index('[PAD]')
    tokens = all_tokens[:pad_loc][1:-1]
    labels = item[-1][:pad_loc][1:-1]
    predicts = sample_results[idx][:pad_loc][1:-1]
    length = len(tokens)
    for i in range(length):
        token = tokens[i]
        label = labels[i]
        predict = idx_to_label[predicts[i]]
        f.write(token+','+label+','+predict+'\n')
    f.write('\n')
f.close()

In [58]:
df = pd.read_csv('./8-9_cause_effec.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './8-9_cause_effec.csv'

In [ ]:
num = len(df)
idx_to_label = {0:"pad", 1:"O", 2:"C", 3:"CE", 4:"E"}
result = []
for i in range(num):
    sentence = df.iloc[i,0]
    tokenizer_result = tokenizer.encode_plus(sentence)
    tokens = tokenizer.tokenize(sentence)
    input_ids = tokenizer_result["input_ids"]
    mask = tokenizer_result["attention_mask"]
    input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
    mask_tensor = torch.LongTensor(mask).unsqueeze(0).to(device)
    output = model(input_tensor, mask_tensor)
    pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
    pred_label = list(map(lambda x:idx_to_label[x], pred))
    result.append([tokens, pred_label[1:-1]])
    

In [ ]:
f = open('./8-9_cause_effec_result.csv', 'w', encoding='utf-8-sig')
for item in result:
    length = len(item[0])
    for i in range(length):
        f.write(item[0][i]+','+item[1][i]+'\n')
    f.write('\n')
f.close()